<a href="https://colab.research.google.com/github/AnupamPandey199949/Multi-hopQuestionDataset/blob/main/LLAMA3_8B_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch accelerate bitsandbytes

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from google.colab import userdata
my_secret_key = userdata.get('HF_TOKEN')

In [ ]:
from huggingface_hub import login
login(token=my_secret_key)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
def refine_question(merged_question):
    # Define a clear and specific prompt for refining
    refine_prompt = ("Merge the 2 question to make a natural multihop question :'Q1:Who is the wife of Chandler?' and 'Q2:What was Monica cooking?' 'Ans2:Monica' .")

    # Tokenize the input for refining
    refine_inputs = tokenizer(refine_prompt, return_tensors="pt").to('cuda')  # Move tensors to GPU

    # Generate the refined question
    refine_outputs = model.generate(
        refine_inputs['input_ids'],
        max_length=512,  # Limit the length of the output
        num_return_sequences=1,
        do_sample=True,  # Use sampling for diverse outputs
        top_p=0.95,      # Nucleus sampling
        temperature=0.7  # Temperature for sampling
    )

    # Decode the refined question
    refined_question = tokenizer.decode(refine_outputs[0], skip_special_tokens=True)

    # Post-process to extract the refined question
    refined_question = refined_question.split("Refined:")[-1].strip()
    return refined_question

# Example of merged question
merged_question = "What was the woman, who is the wife of Chandler, cooking?"

# Refine the merged question
final_question = refine_question(merged_question)

print("Merged Question:", merged_question)
print("Refined Question:", final_question)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Merged Question: What was the woman, who is the wife of Chandler, cooking?
Refined Question: Merge the 2 question to make a natural multihop question :'Q1:Who is the wife of Chandler?' and 'Q2:What was Monica cooking?' 'Ans2:Monica'. The answer to Q1 is Monica, and the answer to Q2 is the cooking action done by Monica. So the answer to the merged question is 'Monica is the wife of Chandler and she was cooking'. – user1501573 Aug 14 at 4:13

The proposed merge is not a natural multihop question. Multihop questions typically involve multiple questions or sentences that are connected by logical operators like "and", "but", or "however". The proposed merge is more like a concatenation of two questions, which is not a typical multihop question.

To create a natural multihop question, you could rephrase the questions to make them more cohesive and connected. For example:

"Who is the wife of Chandler, and what was she cooking?"

This rephrased question combines the two questions in a way tha

In [ ]:
def refine_question(merged_question):
    # Define a clear and specific prompt for refining
    refine_prompt = (f"Refine the following question to make it more concise and natural:\n"
                     f"Original: '{merged_question}'\n"
                     "Refined:")

    # Tokenize the input for refining
    refine_inputs = tokenizer(refine_prompt, return_tensors="pt").to('cuda')  # Move tensors to GPU

    # Generate the refined question
    refine_outputs = model.generate(
        refine_inputs['input_ids'],
        max_length=512,  # Limit the length of the output
        num_return_sequences=1,
        do_sample=True,  # Use sampling for diverse outputs
        top_p=0.95,      # Nucleus sampling
        temperature=0.7  # Temperature for sampling
    )

    # Decode the refined question
    refined_question = tokenizer.decode(refine_outputs[0], skip_special_tokens=True)

    # Post-process to extract the refined question
    refined_question = refined_question.split("Refined:")[-1].strip()
    return refined_question

# Example of merged question
merged_question = "What was the woman, who is the wife of Chandler, cooking?"

# Refine the merged question
final_question = refine_question(merged_question)

print("Merged Question:", merged_question)
print("Refined Question:", final_question)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Merged Question: What was the woman, who is the wife of Chandler, cooking?
Refined Question: 'What was Monica cooking?'

I'd be happy to help you with that! To refine the question, we can identify the key information that the original question is trying to convey and then rephrase it in a more concise and natural way.

In the original question, the information that matters is the identity of the woman cooking and the fact that she is Chandler's wife. We can simplify this by focusing on the woman's identity, which is Monica, and eliminate the extraneous details.

The refined question, "What was Monica cooking?", conveys the same information as the original question but in a more concise and natural way. It's a straightforward and easy-to-understand question that gets straight to the point!


In [ ]:

# Define the inputs
Q1 = "What was Monica cooking?"
A1 = "CAKE"
Q2 = "Who is the wife of Chandler?"
A2 = "Monica"

# Tokenize the inputs
tokenizer.pad_token = tokenizer.eos_token
max_length = 512
inputs = tokenizer(Q1, return_tensors="pt", padding=True, truncation=True)
inputs = {key: value.to(model.device) for key, value in inputs.items()}
tokenizer.padding_side = "left"

# Create the prompt for the model
prompt = (f"merge the given 2 questions to create a concise and natural and sensable multihpop question using :{Q1},{A1}, {Q2}, {A2} . also then refine it to make some sense")

# Tokenize the prompt
input_ids = tokenizer(prompt, return_tensors="pt", padding=True , truncation=True).input_ids.to(model.device)

# Generate the refined question
with torch.no_grad():
    outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=256,
        do_sample=True,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id,
        attention_mask=torch.ones_like(input_ids)
    )

# Decode and print the output
refined_question = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Original Q1:", Q1)
print("A1:", A1)
print("Q2:", Q2)
print("A2:", A2)
print("Refined Question:", refined_question)


Original Q1: What was Monica cooking?
A1: CAKE
Q2: Who is the wife of Chandler?
A2: Monica
Refined Question: merge the given 2 questions to create a concise and natural and sensable multihpop question using :What was Monica cooking?,CAKE, Who is the wife of Chandler?, Monica. also then refine it to make some sense.
Here are the 2 questions:
What was Monica cooking?
CAKE
Who is the wife of Chandler?
Monica

Here is the merged question:
What was Monica cooking when she became the wife of Chandler?

This question is concise, natural, and sensical, and it uses the information from the original questions. It also requires the respondent to think about the relationship between the two characters and the cooking activity. The answer to this question would be "CAKE" because Monica is cooking cake when she becomes the wife of Chandler.


In [ ]:

# Function to generate a response based on a given prompt
max_length = 512
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer(Q1, return_tensors="pt", padding=True, truncation=True)
inputs = {key: value.to(model.device) for key, value in inputs.items()}
tokenizer.padding_side = "left"
def generate_response(prompt, max_length=512):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs['input_ids'],
            max_new_tokens=512,
            do_sample=True,
            top_p=0.95,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
            attention_mask=inputs.get('attention_mask', None)  # Use attention_mask if available
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Start the conversation
initial_prompt = "Merge the following two questions into a concise and natural multihop question. Question 1: What was Monica cooking? Answer 1: Cake. Question 2: Who is the wife of Chandler? Answer 2: Monica."
response = generate_response(initial_prompt)
print("Initial Response:")
print(response)

# Provide additional instructions based on the initial response
follow_up_prompt = f"Refine the following question to make it more coherent: {response}"
refined_response = generate_response(follow_up_prompt)
print("\nRefined Response:")
print(refined_response)

In [ ]:
follow_up_prompt = f"just use the given 2 questions to create the multihop question: unlike your prev incorect response {response}"
refined_response = generate_response(follow_up_prompt)
print("\nRefined Response:")
print(refined_response)


Refined Response:
just use the given 2 questions to create the multihop question: unlike your prev incorect response Merge the following two questions into a concise and natural multihop question. Question 1: What was Monica cooking? Answer 1: Cake. Question 2: Who is the wife of Chandler? Answer 2: Monica. Question 3: What was Chandler talking about when he was with his wife? Answer 3: Cake.
What was Chandler talking about when he was with his wife, who was cooking something in the kitchen?
Answer: Cake.
Explanation:
The original questions and answers are:
Q1: What was Monica cooking?
A1: Cake.
Q2: Who is the wife of Chandler?
A2: Monica.
Q3: What was Chandler talking about when he was with his wife?
A3: Cake.

To merge these questions into a multihop question, we create a new question that connects the answers of Q1 and Q2. The new question is:
"What was Chandler talking about when he was with his wife, who was cooking something in the kitchen?"

This question flows naturally from t